In [13]:
from zipfile import ZipFile 
  
# loading the temp.zip and creating a zip object 
with ZipFile("/scratchB/oqcardoso/predict_motion/unerasedTestset.zip", 'r') as zObject: 
  
    # Extracting specific file in the zip 
    # into a specific location. 
    zObject.extractall( path="") 
zObject.close() 

In [28]:
import pandas as pd
import re
import os

truth = pd.read_csv('./unerasedTestset/index-test.csv', header=None)
truth.columns = ["folder", "datatype", "truth"]
truth.drop("datatype", axis=1, inplace=True)
truth["data_type"] = truth["folder"].apply(lambda x: re.search(r'files/\d+/(.*)/.*', x).group(1))
truth["brief"] = truth["folder"].apply(lambda x: re.search(r'files/\d+/.*/(.*)', x).group(1))
truth["folder"] = truth["folder"].apply(lambda x: int(re.search(r'files/(\d+)/.*', x).group(1)))
truth.drop_duplicates( keep='first', inplace=True)

In [7]:
truth

,folder,truth,data_type,brief
0,100,grant,support,gov.uscourts.nysd.316653.116.0.txt
1,100,grant,support,gov.uscourts.nysd.316653.117.0.txt
2,100,grant,support,gov.uscourts.nysd.316653.124.0.txt
3,100,grant,support,gov.uscourts.nysd.316653.115.0.txt
4,100,grant,opposition,gov.uscourts.nysd.316653.133.0.txt
...,...,...,...,...
600,990,deny,opposition,gov.uscourts.arwd.28923.10.0.txt
601,993,deny,support,gov.uscourts.txnd.213990.7.0.txt
602,993,deny,opposition,gov.uscourts.txnd.213990.8.0.txt
603,994,deny,support,gov.uscourts.vaed.384803.8.0.txt


In [50]:
# how to iterate over the files in a folder

import numpy as np
import pandas as pd
import re
import cv2

def get_files_in_folder(folder):
    files = []
    for file in os.listdir(folder):
        path = os.path.join(folder, file)
        
    return files

0       100
5      1001
8      1002
10     1003
14     1004
       ... 
594     976
596     980
598     990
601     993
603     994
Name: folder, Length: 250, dtype: int64

In [27]:
folder = "./summaries-predictions/"
for file in os.listdir(folder):
    path = os.path.join(folder, file)

    prediction =  pd.read_csv(path)

    column_type = prediction.columns[0]

    if column_type == "folder":
        temp = truth[['folder', 'truth']]
        temp = temp.drop_duplicates( keep='first')

    else:
    
        if "supp" in path:
            temp = truth[['brief', 'truth', 'data_type']]
            temp = temp[temp["data_type"] == "support"]
        else:
            temp = truth[['brief', 'truth', 'data_type']]
            temp = temp[temp["data_type"] == "opposition"]

    #join truth and prediction via folder column
    prediction.rename(columns={"truth":"target"} , inplace=True)

    merge_df = prediction.merge(temp, on=column_type, how='left')
    print(column_type)
    new_df = merge_df[[column_type, 'predict' if column_type == 'brief' else 'prediction' , 'score', 'truth']]
    new_df.to_csv(path, index=False)


brief
brief
brief
brief
brief
brief
brief
folder
folder
brief
brief
brief
brief
folder
brief
folder
brief
brief
brief
brief
brief
brief
brief
folder
folder
brief
brief
brief


In [25]:
merge_df

,folder,prediction,score,target,truth
0,879,grant,0.764808,deny,grant
1,1106,deny,0.047946,deny,deny
2,976,grant,0.977491,deny,grant
3,221,deny,0.218024,deny,grant
4,880,grant,0.984676,deny,deny
...,...,...,...,...,...
245,1156,deny,0.003900,deny,deny
246,715,deny,0.358828,deny,deny
247,566,deny,0.000143,deny,deny
248,541,deny,0.027577,deny,deny


In [36]:
testset =  pd.read_csv('./testfiles/testset.csv', index_col=0)
testset

,prompt,completion,brief_type,data_type,file_path,file_name
5,Case 1:01-cv-00109-SSB\n\nDocument 56\n\nFiled...,TBD,support,test,879,gov.uscourts.ohsd.2137.56.0.txt
6,Case 1:01-cv-00109-SSB\n\nDocument 57\n\nFiled...,TBD,opposition,test,879,gov.uscourts.ohsd.2137.57.0.txt
9,Case: 3:99-cv-00475-bbc Document #: 90 Filed: ...,TBD,support,test,1106,gov.uscourts.wiwd.2665.90.0.txt
10,Case: 3:99-cv-00475-bbc Document #: 101 Filed:...,TBD,opposition,test,1106,gov.uscourts.wiwd.2665.101.0.txt
11,Case: 3:99-cv-00475-bbc Document #: 106 Filed:...,TBD,opposition,test,1106,gov.uscourts.wiwd.2665.106.0.txt
...,...,...,...,...,...,...
1195,Case 3:07-cv-02000-H-CAB\n\n1\n2\n3\n4\n\nDocu...,TBD,opposition,test,566,gov.uscourts.casd.256908.56.0.txt
1206,UNITED STATES DISTRICT COURT\nFOR THE DISTRICT...,TBD,support,test,541,gov.uscourts.mad.127565.28.0.txt
1207,IN THE UNITED STATES DISTRICT COURT\nFOR THE D...,TBD,opposition,test,541,gov.uscourts.mad.127565.40.0.txt
1213,-\n\n \n\noO ODO Oo YH Oo OO FSF W ND\n\nano o...,TBD,support,test,835,gov.uscourts.casd.258578.50.0.txt


In [31]:
paired_testset =  pd.read_csv('./unerasedTestset/paired_testset.csv', index_col=0)
paired_testset

,support,support_file,opposition,opposition_file,outcome,folder_id,case_path,data_type
0,"['CLRB Hanson Industries, LLC et al v. Google ...",['gov.uscourts.cand.34465.234.0.txt'],"['CLRB Hanson Industries, LLC et al v. Google ...",['gov.uscourts.cand.34465.288.0.txt'],deny,456,/mnt/a536sing/sdr1/gvc-recap/5/gov.uscourts.ca...,train
1,['Case 1:08-cv-06978-TPG Document 586 Filed 07...,"['gov.uscourts.nysd.330523.586.0.txt', 'gov.us...",['Case 1:08-cv-06978-TPG Document 591 Filed 07...,['gov.uscourts.nysd.330523.591.0.txt'],deny,895,/mnt/a536sing/sdr1/gvc-recap/3/gov.uscourts.ny...,train
2,['Case 1:01-cv-00109-SSB\n\nDocument 56\n\nFil...,['gov.uscourts.ohsd.2137.56.0.txt'],['Case 1:01-cv-00109-SSB\n\nDocument 57\n\nFil...,['gov.uscourts.ohsd.2137.57.0.txt'],grant,879,/mnt/a536sing/sdt1/gvc-recap/7/gov.uscourts.oh...,test
3,"[""Case 1:10-cv-01736-RMC Document 20\n\nFiled ...",['gov.uscourts.dcd.144464.20.0.txt'],"[""Case 1:10-cv-01736-RMC Document 21\n\nFiled ...",['gov.uscourts.dcd.144464.21.0.txt'],grant,384,/mnt/a536sing/sdm1/gvc-recap/4/gov.uscourts.dc...,train
4,['Case: 3:99-cv-00475-bbc Document #: 90 Filed...,['gov.uscourts.wiwd.2665.90.0.txt'],['Case: 3:99-cv-00475-bbc Document #: 101 File...,"['gov.uscourts.wiwd.2665.101.0.txt', 'gov.usco...",deny,1106,/mnt/a536sing/sdah1/gvc-recap/5/gov.uscourts.w...,test
...,...,...,...,...,...,...,...,...
495,['Case 4:09-cv-02651 Document 19 Filed in TXSD...,['gov.uscourts.txsd.691059.19.0.txt'],['Case 4:09-cv-02651 Document 23\n\nFiled in T...,['gov.uscourts.txsd.691059.23.0.txt'],grant,349,/mnt/a536sing/sdh1/gvc-recap/9/gov.uscourts.tx...,train
496,['UNITED STATES DISTRICT COURT\nFOR THE DISTRI...,['gov.uscourts.mad.127565.28.0.txt'],"[""IN THE UNITED STATES DISTRICT COURT\nFOR THE...",['gov.uscourts.mad.127565.40.0.txt'],deny,541,/mnt/a536sing/sdc1/gvc-recap/5/gov.uscourts.ma...,test
497,"[""..\nCLERK'S OFFICE U.S. DIST. COURT\nAT ROAN...",['gov.uscourts.vawd.108351.5.0.txt'],['IN THE UNITED STATES DISTRICT COURT\nFOR THE...,['gov.uscourts.vawd.108351.10.0.txt'],grant,815,/mnt/a536sing/sdah1/gvc-recap/1/gov.uscourts.v...,train
498,['Case 3:07-cv-02000-H-CAB\n\n1\n2\n3\n\nDocum...,['gov.uscourts.casd.256908.58.0.txt'],['Case 3:07-cv-02000-H-CAB\n\n1\n2\n3\n4\n5\n6...,"['gov.uscourts.casd.256908.95.0.txt', 'gov.usc...",deny,892,/mnt/a536sing/sdt1/gvc-recap/8/gov.uscourts.ca...,train


In [ ]:
tfidf = pd.read_csv('motion-script/svm.fil', index_col=0, header=None)
VOCABULARY_LENGTH = 19925

df = pd.DataFrame(columns=['file_name', 'tfidf'])
df['file_name'] = df.index

file_names =[]
vectors = []
for i in range(0, len(tfidf)):
    row = tfidf.iloc[i].name
    file_name = re.search(r'(.*\.txt)', row).group(1)
    file_names.append(file_name)
    vector_indices = re.findall(r'(\d+):(\d+\.\d+)', tfidf.iloc[i].name)
    vector_indices = {int(k): float(v) for k, v in vector_indices}
    vectors.append(vector_indices)

df['file_name'] = file_names
df['tfidf'] = vectors

df = df.set_index('file_name')
testset = testset.set_index('file_name')
testset = testset.join(df, how='inner')

df.head()